In [ ]:
import numpy as np
import tensorflow as tf 
from keras.models import load_model

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten,Subtract,Reshape
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D,Conv2D,MaxPooling2D,Input,Lambda,GlobalMaxPooling2D
from keras.regularizers import l2
from keras import backend as K
from keras.applications.vgg16 import VGG16, ResNet50
from skimage.io import imsave

from matplotlib.pyplot import imread
from skimage.transform import rescale, resize
import os

In [ ]:
input_shape = [224,224,3]
dataset_path = '../../datasets/WhitePhish/'
input_shape = [224,224,3]
cnn = 'VGG16'
pooling = 'avg'

# Load dataset

In [ ]:
def read_imgs_per_website(data_path,targets,imgs_num,reshape_size,start_target_count):
    all_imgs = np.zeros(shape=[imgs_num,224,224,3])
    all_labels = np.zeros(shape=[imgs_num,1])
    
    all_file_names = []
    targets_list = targets.splitlines()
    count = 0
    for i in range(0,len(targets_list)):
        target_path = data_path + targets_list[i]
        print(target_path)
        file_names = sorted(os.listdir(target_path))
        for j in range(0,len(file_names)):
            try:
                img = imread(target_path+'/'+file_names[j])
                img = img[:,:,0:3]
                all_imgs[count,:,:,:] = resize(img, (reshape_size[0], reshape_size[1]),anti_aliasing=True)
                all_labels[count,:] = i + start_target_count
                all_file_names.append(file_names[j])
                count = count + 1
            except:
                #some images were saved with a wrong extensions 
                try:
                    img = imread(target_path+'/'+file_names[j],format='jpeg')
                    img = img[:,:,0:3]
                    all_imgs[count,:,:,:] = resize(img, (reshape_size[0], reshape_size[1]),anti_aliasing=True)
                    all_labels[count,:] = i + start_target_count
                    all_file_names.append(file_names[j])
                    count = count + 1
                except:
                    print('failed at:')
                    print('***')
                    print(file_names[j])
                    break 
    return all_imgs,all_labels,all_file_names

In [ ]:
# Read images legit (train)
data_path = dataset_path + 'legitimate_whitelist/'
targets_file = open(data_path+'targets.txt', "r")
targets = targets_file.read()
imgs_num = 9363
all_imgs_train,all_labels_train,all_file_names_train = read_imgs_per_website(data_path,targets,imgs_num,reshape_size,0)

# Read images phishing
data_path = dataset_path + 'phishing/'
targets_file = open(data_path+'targets.txt', "r")
targets = targets_file.read()
imgs_num = 1195
all_imgs_test,all_labels_test,all_file_names_test = read_imgs_per_website(data_path,targets,imgs_num,reshape_size,0)



# Load model and predict

In [ ]:
if cnn == 'VGG16':
    model = VGG16(weights='imagenet',  input_shape=input_shape, include_top=False, pooling=pooling)
elif cnn == 'ResNet50':
    model = ResNet50(weights='imagenet',  input_shape=input_shape, include_top=False, pooling=pooling)

In [ ]:
X_train_emb = model.predict(all_imgs_train,batch_size=64)
X_phish_emb = model.predict(all_imgs_test,batch_size=64)

np.save('whitelist_'+cnn+'_emb',X_train_emb)
np.save('phishing_'+cnn+'_emb',X_phish_emb )
